In [1]:
import os
print(os.getcwd())

work_directory_R = os.getcwd() + "/Results_ROC"
work_directory_C = os.getcwd() + "/Results_CM"
work_directory_P = os.getcwd() + "/ResultsPredicted"
Results_File = os.getcwd() + "/Metrics_Result.txt"

import shutil

## Try to remove tree; if failed show an error using try...except on screen
try:
    shutil.rmtree(work_directory_R)
    shutil.rmtree(work_directory_C)
    shutil.rmtree(work_directory_P)
    os.remove(Results_File)
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))
    
import gc as g

# %reset
from IPython import get_ipython
get_ipython().magic('reset -sf') 

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)

import tensorflow as tf,keras
from keras import backend as K, optimizers, regularizers
# K.clear_session()

import pickle, sys,os,cv2, numpy as np, pandas as pd, scipy, seaborn as sns, json, joblib, sklearn as sk
from random import shuffle
from keras.models import Sequential, Model, load_model, model_from_json
# from keras.layers import Dense, Permute, Dropout, Activation, Flatten, Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, ZeroPadding2D, merge
from keras.layers import Dense, Dropout, Permute, Activation, Flatten, Conv2D, MaxPool2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, ZeroPadding2D, merge
from keras.layers import SeparableConv2D, DepthwiseConv2D, BatchNormalization, SpatialDropout2D
from keras.layers.merge import concatenate
from keras import utils as np_utils
from keras.regularizers import l2, l1_l2
from keras.constraints import max_norm

from keras.optimizers import Adam,RMSprop,SGD
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator,array_to_img
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, EarlyStopping

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
from skimage import io

get_ipython().magic('matplotlib inline')
%matplotlib inline

from keras import layers
# from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold, GroupKFold, RepeatedKFold, train_test_split, GridSearchCV
from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix, roc_curve, auc, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler

# from keras.utils.training_utils import multi_gpu_model
# from keras.utils import multi_gpu_model
# from keras import layers
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
# from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import joblib

from tqdm import tqdm
from functools import partial
from sklearn import metrics
from collections import Counter
from itertools import cycle
from six.moves import cPickle as pickle
from scipy import interp
from skimage.feature import local_binary_pattern,hog
from skimage import data, exposure

import itertools
from time import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from radiomics import featureextractor  # This module is used for interaction with pyradiomics
import radiomics
# import SimpleITK as sitk
import pkgutil
import xlsxwriter
from PIL import Image
from natsort import natsorted

print('Libraries Loaded')

print()
print(f"Python Version: {sys.version}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"OpenCV Version: {cv2.__version__}")
print(f"Scikit-Learn Version: {sk.__version__}")

tf.__version__
tf.test.is_built_with_cuda()

# GPU Check 1
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


D:\Research Work\Paper - 4\BreastLiverNet\MoNuSAC\My_Code 19-05-2022 Professor Idea\Final Code\Decemeber_22
Error: D:\Research Work\Paper - 4\BreastLiverNet\MoNuSAC\My_Code 19-05-2022 Professor Idea\Final Code\Decemeber_22/Results_ROC - The system cannot find the path specified.


Using TensorFlow backend.


Libraries Loaded

Python Version: 3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]
Tensor Flow Version: 1.15.0
Keras Version: 2.2.4-tf
Numpy Version: 1.19.2
Pandas Version: 1.1.5
OpenCV Version: 4.6.0
Scikit-Learn Version: 0.24.2


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11746872096321789512,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15125499085
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9175309535626112669
 physical_device_desc: "device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5"]

In [2]:
# # print(os.listdir(os.path.dirname(radiomics.__file__)))

# for _, mod, _ in pkgutil.iter_modules([os.path.dirname(radiomics.__file__)]):
#     print(mod)
    

_cmatrices
_cshape
_version
base
featureextractor
features
firstorder
generalinfo
glcm
gldm
glrlm
glszm
imageoperations
ngtdm
scripts
shape
shape2D


In [3]:
def create_directory(directory):
    '''
    Creates a new folder in the specified directory if the folder doesn't exist.
    INPUT
        directory: Folder to be created, called as "folder/".
    OUTPUT
        New folder in the current directory.
    '''
    if not os.path.exists(directory):
        os.makedirs(directory)
        
def read_nuclei(path):
    "Read raw data"

    # Load 4-channel image
    if len(path) == 0:
        return None
    
    img = skimage.io.imread(path)
    
    # input image
    if len(img.shape) > 2:
        img = img[:,:,:3]
    # mask
    else:
        # do nothing
        pass
        
    return img


def save_nuclei(path, img):
    "save image"
    skimage.io.imsave(path, img)

In [4]:
################################# Old Technqiue ##################################
patch_size = (128, 128)
step = 200

data_folder_R = "D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/"

print('\nSelect Organ')
option_Organ = int(input("\n 1-Breast \n 2-Kidney \n 3-Lung \n 4-Prostate \n"))
while option_Organ  not in (1,2,3,4,5):
    option_Organ = int(input("\n 1-Breast \n 2-Kidney \n 3-Lung \n 4-Prostate \n"))

num_classes=4
loss_a = 'categorical_crossentropy'

if option_Organ==1:
    Organ = "Breast"
    Organ_ = "Breast_I"
elif option_Organ==2:
    Organ = "Kidney"
    Organ_ = "Kidney_I"
elif option_Organ==3:
    Organ = "Lung"
    Organ_ = "Lung_I"
elif option_Organ==4:
    Organ = "Prostate"
    Organ_ = "Prostate_I"


print('\nSelect Train or Test')
option_Mode = int(input("\n 1-Train \n 2-Test \n"))
while option_Mode  not in (1,2):
    option_Mode = int(input("\n 1-Train \n 2-Test \n"))


print("\n\n\n Organ Type: ", Organ)

if option_Mode==1:
    Mode = "Train"
    path_ = "D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Select_January_23/Training_Data_Classwise_" + str(patch_size[0]) + "_" + str(step) + "/Data_Patches/" + str(Organ)


    
# elif option_Mode==2:
#     Mode = "Test"
#     path_ = "D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Select_January_23/Testing_Data_Classwise_" + str(patch_size[0]) + "_" + str(step) + "/Data_Patches/" + str(Organ) 

data_folder1 = "/Images/"
path1  = path_ + data_folder1
folder1 = sorted(os.listdir(path1))
print('\nTotal Images: ', len(folder1))

data_folder2 = "/Masks/"
path2  = path_ + data_folder2
folder2 = sorted(os.listdir(path2))
print('Total Masks: ', len(folder2))

folder_diff = [file for file in folder1 if file.endswith(".png")]
folder_label = [file for file in folder2 if file.endswith(".png")]

# print(folder_diff)
# print(folder_label)

print(path1)
print(path2)

print(len(folder_diff))
print(len(folder_label))

# New
IMAGES_DEST_S =  os.path.join(path_, "Images/")
MASKS_DEST_S = os.path.join(path_, "Masks/")

# Make new folders
try:
    # Make directory
    create_directory(IMAGES_DEST_S)
except OSError:
    print ("Creation of the mask directory %s failed" % IMAGES_DEST_S)


try:
    # Make directory
    create_directory(MASKS_DEST_S)
except OSError:
    print ("Creation of the mask directory %s failed" % MASKS_DEST_S)
    

        


Select Organ

 1-Breast 
 2-Kidney 
 3-Lung 
 4-Prostate 
4

Select Train or Test

 1-Train 
 2-Test 
1



 Organ Type:  Prostate

Total Images:  46
Total Masks:  46
D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Select_January_23/Training_Data_Classwise_128_200/Data_Patches/Prostate/Images/
D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Select_January_23/Training_Data_Classwise_128_200/Data_Patches/Prostate/Masks/
46
46


In [5]:

def load_pickle(data_file,q):
    print('\nTring to load pickle from %s' % data_file)
    with open(data_file, 'rb') as file:
        datasets = pickle.load(file)
        dataset = datasets['dataset']

    X_train = dataset['X_train']
    Y_train = dataset['Y_train']
    
    print('\nPickle Loaded Successfully!')
    
    del dataset

    if q==1:
        print('\nX_train shape:', X_train.shape)
        print('Y_train shape:', Y_train.shape)
    
    return X_train,Y_train



def load_pickle__(data_file,q):
    print('\nTring to load pickle from %s' % data_file)
    with open(data_file, 'rb') as file:
        datasets = pickle.load(file)
        dataset = datasets['dataset']

    X_train = dataset['X_train']
    Y_train = dataset['Y_train']
    Y_train_ = dataset['Y_train_']
    
    print('\nPickle Loaded Successfully!')
    
    del dataset

    if q==1:
        print('\nX_train shape:', X_train.shape)
        print('Y_train shape:', Y_train.shape)
        print('Y_train shape:', Y_train_.shape)
    
    return X_train,Y_train,Y_train_



def save_pickle(x_train,y_train, data_file):
    print('\nTrying to save pickle to %s' % data_file)
    
    X_train = x_train
    Y_train = y_train
    del x_train,y_train
    
    # creating dictionary to store trian and test data
    datasets = {'dataset' : {'X_train': X_train,'Y_train': Y_train}}

    with open(data_file, 'wb') as file:
        pickle.dump(datasets, file, protocol=pickle.HIGHEST_PROTOCOL)
        del datasets # to free up memory.
        
    print('\nPickle Saved Successfully!')



def save_pickle__(x_train,y_train,y_train_, data_file):
    print('\nTrying to save pickle to %s' % data_file)
    
    X_train = x_train
    Y_train = y_train
    Y_train_ = y_train_
    del x_train,y_train,y_train_
    
    # creating dictionary to store trian and test data
    datasets = {'dataset' : {'X_train': X_train,'Y_train': Y_train,'Y_train_': Y_train_}}

    with open(data_file, 'wb') as file:
        pickle.dump(datasets, file, protocol=pickle.HIGHEST_PROTOCOL)
        del datasets # to free up memory.
        
    print('\nPickle Saved Successfully!')
    

In [6]:
# Augment-September-2022--
if option_Mode==1:
    if option_Organ==1:
        folder_diff_ = 52
    elif option_Organ==2:
        folder_diff_ = 113
    elif option_Organ==3:
        folder_diff_ = 89
    elif option_Organ==4:
        folder_diff_ = 46


In [7]:
print(IMAGES_DEST_S)
print(MASKS_DEST_S)


D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Select_January_23/Training_Data_Classwise_128_200/Data_Patches/Prostate\Images/
D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Select_January_23/Training_Data_Classwise_128_200/Data_Patches/Prostate\Masks/


In [8]:
if Mode == "Train":
    resultDir_texture = os.path.join(data_folder_R, "Augment_January_23", "Result_Train/")
    resultDir_texture_Labels = os.path.join(data_folder_R, "Augment_January_23", "Result_Train_Labels/")
elif Mode == "Test":
    resultDir_texture = os.path.join(data_folder_R, "Augment_January_23", "Result_Test/")
    resultDir_texture_Labels = os.path.join(data_folder_R, "Augment_January_23", "Result_Test_Labels/")



df_texture_New = pd.DataFrame()
df_shape_New = pd.DataFrame()
df_labels = np. array([])

resultExcelPath_Labels = os.path.join(resultDir_texture_Labels,  str(Organ) + ".csv")

df_texture_Labels = pd.DataFrame()
df_texture_Labels = pd.read_csv(resultExcelPath_Labels, header=None, sep=',', index_col=False)
df_texture_Labels = df_texture_Labels.T


X_train_S_ = []
Y_train_S_ = []

print("\n\t\t\t\t===== Processing Image =====")
k = 0

for i in range(1, folder_diff_):
    print("\n\n\nImage #:", str(i))
    resultExcelPath_texture = os.path.join(resultDir_texture,  str(Organ), str(i) + ".csv")
    print("Input: ", resultExcelPath_texture)
    print("Labels: ", resultExcelPath_Labels)

#     Loop the data lines
    with open(resultExcelPath_texture, 'r') as temp_f:
        # get No of columns in each line
        col_count = [ len(l.split(",")) for l in temp_f.readlines() ]

    column_names = [i for i in range(0, max(col_count))]

#     Read csv
    df_texture = pd.read_csv(resultExcelPath_texture, delimiter=",", names=column_names, index_col=False)    
    df_texture.drop(df_texture.columns[0], axis=1, inplace=True)        
    df_texture = df_texture.T

    df_texture.drop(df_texture.columns[0], axis=1, inplace=True)        

    Size = df_texture.shape[0]
    print("Total Files Input:", Size)
    print("Total Files Label:", df_texture_Labels.values[k,0], "\n")
    
    imgResult = read_nuclei(os.path.join(path1,folder_diff[i]))
    maskResult = read_nuclei(os.path.join(path2,folder_label[i]))
    
    for i_S in range(df_texture_Labels.values[k,0]):
        temp = os.path.splitext(folder_diff[i])[0]
        save_nuclei(IMAGES_DEST_S + "{}_Extra-{}.png".format(temp,i_S), imgResult)
        
        temp = os.path.splitext(folder_label[i])[0]
        save_nuclei(MASKS_DEST_S + "{}_Extra-{}.png".format(temp,i_S), maskResult)
    
        X_train_S_.append(imgResult)
        Y_train_S_.append(maskResult)
                    

    df_texture_New = df_texture_New.append(df_texture, ignore_index = True)
    print("Texture :", df_texture_New.shape)

    df_labels = np. append(df_labels, df_texture_Labels.values[k,0])
    print("Labels :", df_labels.shape)
    k = k + 1

    del df_texture, resultExcelPath_texture
    
    
    


				===== Processing Image =====



Image #: 1
Input:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train/Prostate\1.csv
Labels:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train_Labels/Prostate.csv
Total Files Input: 1
Total Files Label: 1 

Texture : (1, 107)
Labels : (1,)



Image #: 2
Input:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train/Prostate\2.csv
Labels:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train_Labels/Prostate.csv
Total Files Input: 34
Total Files Label: 34 

Texture : (35, 107)
Labels : (2,)



Image #: 3
Input:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train/Prostate\3.csv
Labels:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train_Labels/Prostate.csv
Total Files Input: 49
Total Files Label: 49 

Texture : (84, 107)
Labels : (3,)



Image #: 4
Input:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_

Texture : (930, 107)
Labels : (32,)



Image #: 33
Input:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train/Prostate\33.csv
Labels:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train_Labels/Prostate.csv
Total Files Input: 11
Total Files Label: 11 

Texture : (941, 107)
Labels : (33,)



Image #: 34
Input:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train/Prostate\34.csv
Labels:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train_Labels/Prostate.csv
Total Files Input: 2
Total Files Label: 2 

Texture : (943, 107)
Labels : (34,)



Image #: 35
Input:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train/Prostate\35.csv
Labels:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Augment_January_23\Result_Train_Labels/Prostate.csv
Total Files Input: 1
Total Files Label: 1 

Texture : (944, 107)
Labels : (35,)



Image #: 36
Input:  D://Taimoor_Datasets/Thesis/Nuclei/MoNuSAC/Au

In [9]:
print("\nTexture :", df_texture_New.shape)
print("Labels :", df_labels.shape)
print("Labels :", df_labels)

data_file = 'January/PICKLESNEW/Classify_B_' + str(Mode) + '_' + str(Organ) + '.pickle'   
data_file_ = 'January/PICKLESNEW/Classify_B_' + str(Mode) + '_' + str(Organ_) + '.pickle'   
# print(data_file)

X_train_Pickle, Y_train_Pickle = load_pickle(data_file,1)

print("\nLabels Nuclei's Shape :", Y_train_Pickle.shape)
print("Labels Nuclei's:", Y_train_Pickle)



Texture : (975, 107)
Labels : (45,)
Labels : [ 1. 34. 49. 93. 64. 67. 54.  8.  1. 21.  1. 49. 93. 64. 67. 54.  8.  1.
 21.  1.  1.  2.  4. 36. 49. 46. 28.  4.  2.  3.  1.  3. 11.  2.  1.  5.
  3.  3.  3.  2.  1.  5.  3.  3.  3.]

Tring to load pickle from January/PICKLESNEW/Classify_B_Train_Prostate.pickle

Pickle Loaded Successfully!

X_train shape: (45, 128, 128, 3)
Y_train shape: (45,)

Labels Nuclei's Shape : (45,)
Labels Nuclei's: [1 3 0 0 0 1 1 1 2 2 2 0 0 0 1 1 1 2 2 2 0 1 3 0 0 0 0 1 2 0 1 1 3 1 1 2 2
 2 3 1 1 2 2 2 3]


In [10]:
print("Texture :", df_texture_New.shape)
print("Labels :", df_labels.shape)

Total_Lables = np.cumsum(df_labels)
Total_Lables = np.sort(np.append (Total_Lables, 0))

X_train_ = df_texture_New.to_numpy() # format is in (trials, channels, samples)
Y_train_ = np.zeros(df_texture_New.shape[0])

for i in range(len(Total_Lables)-1):
    a = Total_Lables[i] 
    b = Total_Lables[i+1]
    Y_train_[int(a):int(b)] = Y_train_Pickle[i]

Y_train_ = np.asarray(Y_train_)

print('\nX_train shape:', X_train_.shape)
print('Y_train shape:', Y_train_.shape)
print('Y_train :', Y_train_)



Texture : (975, 107)
Labels : (45,)

X_train shape: (975, 107)
Y_train shape: (975,)
Y_train : [1. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [11]:
X_train_S = np.asarray(X_train_S_)
Y_train_S = np.asarray(Y_train_S_)

print('\nX_train shape:', X_train_S.shape)
print('Y_train shape:', Y_train_S.shape)



X_train shape: (975, 128, 128, 3)
Y_train shape: (975, 128, 128)


In [12]:
def Mode(Organ, work_directory_Pickles, option_Mode):

    if option_Mode==1:
        Mode = "Train"
        data_file1 = work_directory_Pickles + '/Classify_C_' + str(Mode) + '_' + str(Organ) + '.pickle'
        data_file2 = ""
        
    elif option_Mode==2:
        Mode = "Test"
        data_file1 = work_directory_Pickles + '/Classify_C_' + str(Mode) + '_' + str(Organ) + '.pickle'
        data_file2 = ""
        
    elif option_Mode==3:
        Mode = "Train"
        data_file1 = work_directory_Pickles + '/Classify_C_' + str(Mode) + '_' + str(Organ) + '.pickle'

        Mode = "Test"
        data_file2 = work_directory_Pickles + '/Classify_C_' + str(Mode) + '_' + str(Organ) + '.pickle'
  
    return data_file1, data_file2 


In [14]:
work_directory_Pickles = os.path.join(os.getcwd() + "/January/PICKLESNEW")

print('\nSelect DataSet')
print('\nSelect Save or Load')
option_D0 = int(input("\n 1-Save \n 2-Load \n"))
while option_D0  not in (1,2):
    option_D0 = int(input("\n 1-Save \n 2-Load \n"))

num_classes=4
loss_a = 'categorical_crossentropy'

print("\n\n DataSet-MONUSAC \n")
if option_D0==1:
    data_file, _ = Mode(Organ, work_directory_Pickles, option_Mode)
    save_pickle(X_train_, Y_train_, data_file)
    print('\nSave Complete')

    data_file_, _ = Mode(Organ_, work_directory_Pickles, option_Mode)
    save_pickle__(X_train_S, Y_train_S, Y_train_, data_file_)    
    print('\nSave Complete')
    
    
elif option_D0==2:
    data_file, _ = Mode(Organ, work_directory_Pickles, option_Mode)
    Concat_Organs_Output, Concat_Organs_Labels = load_pickle(data_file,1)
    print('\nLoad Complete')
    
    print("\n Labels Portion \n")
    unique, counts = np.unique(Concat_Organs_Labels, return_counts=True)
    abc= dict(zip(unique, counts))
    print(abc)
    for i in range(len(abc)):
        print('Class %d: %d' % (i, abc[i]))

        
        
    data_file_, _ = Mode(Organ_, work_directory_Pickles, option_Mode)
    Concat_Organs_Output_, Concat_Organs_Labels_, Concat_Organs_Labels_S = load_pickle__(data_file_,1)
    
    print('\nLoad Complete')
    
    print("\n Labels Portion \n")
    unique, counts = np.unique(Concat_Organs_Labels_S, return_counts=True)
    abc= dict(zip(unique, counts))
    print(abc)
    for i in range(len(abc)):
        print('Class %d: %d' % (i, abc[i]))

        
        


Select DataSet

Select Save or Load

 1-Save 
 2-Load 
2


 DataSet-MONUSAC 


Tring to load pickle from D:\Research Work\Paper - 4\BreastLiverNet\MoNuSAC\My_Code 19-05-2022 Professor Idea\Final Code\Decemeber_22/January/PICKLESNEW/Classify_C_Train_Prostate.pickle

Pickle Loaded Successfully!

X_train shape: (975, 107)
Y_train shape: (975,)

Load Complete

 Labels Portion 

{0.0: 575, 1.0: 275, 2.0: 70, 3.0: 55}
Class 0: 575
Class 1: 275
Class 2: 70
Class 3: 55

Tring to load pickle from D:\Research Work\Paper - 4\BreastLiverNet\MoNuSAC\My_Code 19-05-2022 Professor Idea\Final Code\Decemeber_22/January/PICKLESNEW/Classify_C_Train_Prostate_I.pickle

Pickle Loaded Successfully!

X_train shape: (975, 128, 128, 3)
Y_train shape: (975, 128, 128)
Y_train shape: (975,)

Load Complete

 Labels Portion 

{0.0: 575, 1.0: 275, 2.0: 70, 3.0: 55}
Class 0: 575
Class 1: 275
Class 2: 70
Class 3: 55
